In [11]:
import numpy as np
import numpy.linalg as LA

In [12]:
def V_z(y):
  raise NotImplementedError

In [13]:
def T(y):
  raise NotImplementedError

In [14]:
def M_x(y):
  raise NotImplementedError

In [15]:
b2 = 16.07/2
root_c = np.nan 
tip_c = np.nan

def chord_length(y):
    """Returns chord length at spanwise posistion"""
    return root_c + y * (tip_c - root_c)/b2

def airfoil_surface(cls, x):
    """Returns upper and lower coordinate of airfoil surface for a chordwise posistion"""
    raise NotImplementedError
    return z_upper, z_lower

In [16]:
class Panel:
    def __init__(self, point1, point2, thickness):
        """point1 and point2 are coordinates relative to the chord, thickness in mm"""
        self.p1 = point1
        self.p2 = point2
        self.vector = point2 - point1
        self.h = LA.norm(self.vector)
        self.t = thickness/1000
        # z is the vertical distance from the x axis relative to chord
        self.z = (point1 + self.vector/2)[0]

    def A(self, chord):
        return self.h * chord * self.t
    
    def Q_x(self, chord):
        """Returns first moment of area about z=0"""
        return self.z * chord * self.area(chord)
    
    def I_xc(self, chord):
        """Returns second moment of area about own axis"""
        cos_a = self.vector[0] / self.h
        sin_a = self.vector[1] / self.h

        return self.t * self.h * (self.t**2 * cos_a**2 + self.h**2 * sin_a**2)
    
    def I_xc(self, chord, z_centroid):
        """Returns second moment of area about centroid"""
        d = self.z * chord - z_centroid
        return self.I_xc(chord) + self.area(chord) * d**2
    


In [17]:
class Stringer:
    def __init__(self, area, point, length):
        """area is in square mmm, point is the coordinate realtive to the chord and length is the spanwise length of the stringer"""
        self.A = area * 1e-6
        self.point = point
        self.length = length
    
    def Q_x(self, chord):
        """Returns first moment of area about z=0"""
        return self.A * self.point[1] * chord

    def I_xc(self, chord, z_centroid):
        """Returns second moment of area about centroid"""
        return self.A * (self.point[0] * chord - z_centroid)**2 


In [18]:
# This is not important it just helps the IDE with autocomplete
from typing import Optional
from collections.abc import Collection

In [20]:
class WingBox:

  def __init__(self, front_spar_x, front_spar_t, rear_spar_x, rear_spar_t, upper_panel_t, lower_panel_t, stringers: Optional[Collection[Stringer]]=None):
                
    front_upper_z, front_lower_z = airfoil_surface(front_spar_x)
    front_upper, front_lower = np.array([front_spar_x, front_upper_z]), np.array([front_spar_x, front_lower_z])
    rear_upper_z, rear_lower_z = airfoil_surface(rear_spar_x)
    rear_upper, rear_lower = np.array([rear_spar_x, rear_upper_z]), np.array([rear_spar_x, rear_lower_z])

    self.points = {"front_upper":front_upper, "front_lower":front_lower, "rear_upper":rear_upper, "rear_lower":rear_lower}

    front_spar = Panel(front_lower, front_upper, front_spar_t)
    rear_spar = Panel(rear_lower, rear_upper, rear_spar_t)
    upper_panel = Panel(front_upper, rear_upper, upper_panel_t)
    lower_panel = Panel(front_lower, rear_lower, lower_panel_t)

    self.panels = {"front spar":front_spar, "rear_spar":rear_spar, "upper_panel":upper_panel, "lower_panel":lower_panel} 
  
    self.stringers = stringers

  def I_xc(self, y):
    c = chord_length(y)
    A = 0
    Az = 0
    I_xc = 0

    for panel in self.panels:
      A += panel.A(c)
      Az += panel.Q_x(c)
    
    if self.stringers is not None:
      for stringer in self.stringers:
        if stringer.length < y:
          pass
        A += stringer.A
        Az += stringer.Q_x(c) 
    
    z_centroid = Az / A

    
    

    if self.stringers is not None:
      for stringer in self.stringers:
        if stringer.length < y:
          pass
        I_xc += stringer.I_xc(c, z_centroid)

    return I_xc
  

  def J(self, y):
    # Calculate area inside box
    x_distance = self.points[1][0] - self.points[0][0]
    A = x_distance/2*(self.panels["front spar"].h + self.panels["front spar"].h)

    integral = 0
    for panel in self.panels:
      integral += 1/panel.t
    raise NotImplementedError
    

TypeError: 'ABCMeta' object is not subscriptable